In [1]:
API_KEY = "LQJQ3NBILSTJQ8D4HY4R87TPDESXBA6EH3MENU7SHY56Q04STCJNMNVM6I38B64GQCV9H0A0QS124CK2"


In [ ]:
from curl_cffi.requests.cookies import Cookies

def cookies_to_dict(cookies) -> dict:
    # works for RequestsCookieJar-like objects
    return {c.name: c.value for c in cookies}

def dict_to_cookies(cookie_dict: dict):
    # curl_cffi accepts plain dict cookies=
    return cookie_dict


def serialize_cookies(cookies):
    return [
        {
            "name": c.name,
            "value": c.value,
            "domain": c.domain,
            "path": c.path,
            "secure": c.secure,
            "expires": c.expires,
        }
        for c in cookies.jar
    ]


def deserialize_cookies(cookie_list):
    jar = Cookies()
    for c in cookie_list:
        jar.set(
            c["name"],
            c["value"],
            domain=c["domain"],
            path=c["path"],
            secure=c["secure"],
            expires=c["expires"],
        )
    return jar



def load_bootstrap_tokens(room_id: str) -> dict:
    path = TOKENS_DIR / f"{room_id}.json"
    with open(path) as f:
        return json.load(f)


In [3]:
import pyairbnb

print(pyairbnb.__file__)

/Users/kamingleung/Development/TryOut/Airbnb-Search/pyairbnb/src/pyairbnb/__init__.py


In [4]:
# import pyairbnb

# from scrapingbee_transport import make_scrapingbee_proxy_url

# print(pyairbnb.__file__)

# proxy_url = make_scrapingbee_proxy_url(api_key=API_KEY)

# details = pyairbnb.get_details(
#     room_url="https://www.airbnb.com/rooms/1541419087261094292",
#     currency="USD",
#     language="en",
#     proxy_url=proxy_url,   # <-- ScrapingBee transport
# )


In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pyairbnb
from scrapingbee_transport import make_scrapingbee_proxy_url

proxy_url = make_scrapingbee_proxy_url(api_key=API_KEY)

In [6]:
room_urls = [
    "https://www.airbnb.com/rooms/720984712450297572",
    "https://www.airbnb.com/rooms/28200634",
    "https://www.airbnb.com/rooms/1540308719239432520",  # replace
]

In [7]:

# def fetch_one(url: str):
#     return pyairbnb.get_details(
#         room_url=url,
#         currency="USD",
#         language="en",
#         proxy_url=proxy_url,
#     )

# results = []
# errors = []

# with ThreadPoolExecutor(max_workers=3) as pool:  # ✅ 3 at a time
#     future_to_url = {pool.submit(fetch_one, url): url for url in room_urls}
#     for fut in as_completed(future_to_url):
#         url = future_to_url[fut]
#         try:
#             results.append((url, fut.result()))
#         except Exception as e:
#             errors.append((url, repr(e)))

# print("done:", len(results), "errors:", len(errors))

In [8]:
def get_room_id_from_url(url: str) -> str:
    return int(url.split("?")[0].split("rooms/")[1])

In [9]:
from pyairbnb import details
from pathlib import Path
import json


room_id = get_room_id_from_url(room_urls[0])

data, price_input, cookies = details.get(
    room_urls[0],
    language="en",
    proxy_url=proxy_url,
)

bootstrap_tokens = {
    "product_id": price_input["product_id"],
    "impression_id": price_input["impression_id"],
    "api_key": price_input["api_key"],
    "cookies": cookies,
}


bootstrap_tokens["cookies"] = serialize_cookies(bootstrap_tokens["cookies"])


TOKENS_DIR = Path("tokens")
TOKENS_DIR.mkdir(parents=True, exist_ok=True)

with open(f"tokens/{room_id}.json", "w") as f:
    json.dump(bootstrap_tokens, f)


In [ ]:
from pyairbnb.calendarinfo import get as get_calendar

calendar_data = get_calendar(
    api_key=bootstrap_tokens["api_key"],
    room_id=room_id,
    month=1,
    year=2026,
    proxy_url=proxy_url,
)

calendar_data

In [14]:
from pyairbnb import api
from pyairbnb.calendarinfo import get as get_calendar

calendar_api_key = api.get(proxy_url)  # <-- fetch a valid key via homepage

calendar_api_key

'd306zoyjsyarp7ifhu67rjxn52tv0t20'

In [11]:
room_id

720984712450297572

# Reviews - ScrapingBee 1 credit

In [40]:
import requests, json
from urllib.parse import urlencode, quote

SCRAPINGBEE_KEY = API_KEY
AIRBNB_API_KEY = "d306zoyjsyarp7ifhu67rjxn52tv0t20"

OP_NAME = "StaysPdpReviewsQuery"
REVIEWS_SHA = "609757475b385832e85a229fd554f752530447d7a620e39940ec8f0c30bd32f0"

def build_reviews_url(airbnb_graphql_id: str,
                      limit: int = 24,
                      offset: int = 0,
                      locale: str = "en",
                      currency: str = "USD",
                      sorting: str = "BEST_QUALITY"):
    """
    airbnb_graphql_id: the 'id' field you saw in variables:
        "U3RheUxpc3Rpbmc6731059780954437660" (base64-ish)
    """
    base = f"https://www.airbnb.com/api/v3/{OP_NAME}/{REVIEWS_SHA}"

    variables = {
        "id": airbnb_graphql_id,
        "pdpReviewsRequest": {
            "fieldSelector": "for_p3_translation_only",
            "forPreview": False,
            "limit": int(limit),
            "offset": str(offset),                 # NOTE: string in your request
            "showingTranslationButton": False,
            "first": int(limit),                   # duplicate but present in request
            "sortingPreference": sorting,
            "numberOfAdults": "1",
            "numberOfChildren": "0",
            "numberOfInfants": "0",
            "numberOfPets": "0",
        }
    }

    extensions = {"persistedQuery": {"version": 1, "sha256Hash": REVIEWS_SHA}}

    params = {
        "operationName": OP_NAME,
        "locale": locale,
        "currency": currency,
        "variables": json.dumps(variables, separators=(",", ":")),
        "extensions": json.dumps(extensions, separators=(",", ":")),
    }

    return base + "?" + urlencode(params, quote_via=quote)

def get_reviews_scrapingbee(listing_id: str,
                            airbnb_graphql_id: str,
                            limit: int = 24,
                            offset: int = 0):
    target_url = build_reviews_url(airbnb_graphql_id, limit=limit, offset=offset)

    sb_params = {
        "api_key": SCRAPINGBEE_KEY,
        "url": target_url,
        "render_js": "false",        # ✅ Classic
        "forward_headers": "true",   # ✅ to forward Spb-* headers
    }

    headers = {
        "Spb-Accept": "application/json",
        "Spb-User-Agent": "Mozilla/5.0",
        "Spb-Referer": f"https://www.airbnb.com/rooms/{listing_id}",
        "Spb-X-Airbnb-Api-Key": AIRBNB_API_KEY,
    }

    r = requests.get("https://app.scrapingbee.com/api/v1/", params=sb_params, headers=headers, timeout=60)
    r.raise_for_status()
    return r.json()

# Example using your listing:
listing_id = "731059780954437660"
airbnb_graphql_id = "U3RheUxpc3Rpbmc6NzMxMDU5NzgwOTU0NDM3NjYw"
data = get_reviews_scrapingbee(listing_id, airbnb_graphql_id, limit=24, offset=0)
data

{'data': {'presentation': {'__typename': 'RootPresentationContainer',
   'stayProductDetailPage': {'__typename': 'StayPDPPresentationContainer',
    'reviews': {'__typename': 'PdpReviews',
     'reviews': [{'__typename': 'PdpReviewForP3',
       'collectionTag': None,
       'comments': 'This is a must stay location! It’s so cute and the views are stunning. A great way to disconnect and breath in some fresh air! My only critique is the outdoor space around the airstream was a bit overgrown with weeds.',
       'id': '1575617902938431201',
       'language': 'en',
       'createdAt': '2025-12-13T22:31:30Z',
       'reviewee': {'__typename': 'ReviewUser',
        'deleted': False,
        'firstName': 'Mark',
        'hostName': 'Mark',
        'id': '443181856',
        'contextualUserId': '1463714340064618214',
        'pictureUrl': 'https://a0.muscache.com/im/pictures/user/User-443181856/original/0fa1b09a-ee7e-462a-b866-ea0fcecf412e.jpeg',
        'profilePath': '/users/profile/146371

# ScrapingBee Calendar  - 1 credit

In [37]:
import requests, json
from urllib.parse import urlencode, quote

SCRAPINGBEE_KEY = API_KEY
AIRBNB_API_KEY = "d306zoyjsyarp7ifhu67rjxn52tv0t20"

HASH = "8f08e03c7bd16fcad3c92a3592c19a8b559a0d0855a84028d1163d4733ed9ade"

def build_pdp_availability_url(listing_id, year, month, count=12):
    base = f"https://www.airbnb.com/api/v3/PdpAvailabilityCalendar/{HASH}"
    variables = {"request": {
        "count": count,
        "listingId": str(listing_id),
        "month": int(month),
        "year": int(year),
        "returnPropertyLevelCalendarIfApplicable": False
    }}
    extensions = {"persistedQuery": {"version": 1, "sha256Hash": HASH}}
    params = {
        "operationName": "PdpAvailabilityCalendar",
        "locale": "en",
        "currency": "USD",
        "variables": json.dumps(variables, separators=(",", ":")),
        "extensions": json.dumps(extensions, separators=(",", ":")),
    }
    return base + "?" + urlencode(params, quote_via=quote)

def get_calendar_scrapingbee(listing_id, year, month, count=12):
    target_url = build_pdp_availability_url(listing_id, year, month, count)

    sb_params = {
        "api_key": SCRAPINGBEE_KEY,
        "url": target_url,
        "render_js": "false",          # Classic
        "forward_headers": "true",     # ✅ required to forward headers
    }

    # ✅ Prefix forwarded headers with "Spb-"
    headers = {
        "Spb-Accept": "application/json",
        "Spb-User-Agent": "Mozilla/5.0",
        "Spb-Referer": f"https://www.airbnb.com/rooms/{listing_id}",
        "Spb-X-Airbnb-Api-Key": AIRBNB_API_KEY,
    }

    r = requests.get("https://app.scrapingbee.com/api/v1/", params=sb_params, headers=headers, timeout=60)
    r.raise_for_status()
    return r.json()

data = get_calendar_scrapingbee("731059780954437660", 2025, 12, 12)
print(data.keys())
data

dict_keys(['data', 'extensions'])


{'data': {'merlin': {'__typename': 'MerlinQuery',
   'pdpAvailabilityCalendar': {'__typename': 'MerlinPdpAvailabilityCalendarResponse',
    'calendarMonths': [{'__typename': 'MerlinCalendarMonth',
      'month': 12,
      'year': 2025,
      'days': [{'__typename': 'MerlinCalendarDay',
        'calendarDate': '2025-12-01',
        'available': False,
        'maxNights': 1125,
        'minNights': 1,
        'availableForCheckin': False,
        'availableForCheckout': False,
        'bookable': None,
        'price': {'__typename': 'MerlinCalendarDayPrice',
         'localPriceFormatted': None}},
       {'__typename': 'MerlinCalendarDay',
        'calendarDate': '2025-12-02',
        'available': False,
        'maxNights': 1125,
        'minNights': 1,
        'availableForCheckin': False,
        'availableForCheckout': False,
        'bookable': None,
        'price': {'__typename': 'MerlinCalendarDayPrice',
         'localPriceFormatted': None}},
       {'__typename': 'MerlinCalen

In [33]:
data

{'errors': [{'message': 'Response not successful: Received status code 400',
   'locations': [{'line': 1, 'column': 1}],
   'path': [],
   'extensions': {'response': {'statusCode': 400,
     'body': {'error_code': 400,
      'error': 'invalid_key',
      'error_message': 'Invalid API key.'}},
    'code': 'Bad Request'}}],
 'data': None}

In [12]:
calendar_data = get_calendar(
    api_key=calendar_api_key,
    room_id=str(room_id),
    month=1,
    year=2026,
    proxy_url=proxy_url,
)
calendar_data

top keys: ['errors', 'data']
has errors: True False
FULL ERRORS: [
  {
    "message": "Response not successful: Received status code 400",
    "locations": [
      {
        "line": 1,
        "column": 1
      }
    ],
    "path": [],
    "extensions": {
      "response": {
        "statusCode": 400,
        "body": {
          "error_code": 400,
          "error": "invalid_key",
          "error_message": "Invalid API key."
        }
      },
      "code": "Bad Request"
    }
  }
]


[]